In [1]:
import os
os.chdir("..")

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import codecs
from util.google_search_util import google_the_claim

In [3]:
claim = "Arnold acted in Conan"

In [4]:
pages_to_check = 10

In [5]:
google_query = "+".join(claim.split(" "))

In [6]:
google_query

'Arnold+acted+in+Conan'

In [22]:
pages_to_check=1

In [51]:
wikipedia_titles = set()
google_query = "+".join(claim.split(" "))
for i in range(pages_to_check):
    start = i * 10
    
    page = requests.get(f"https://www.google.com/search", params={"q": google_query, "start": start})
    soup = BeautifulSoup(page.content, "html.parser")
    
    for div in soup.find_all("div", class_="g"):
        if div.find("h3") is None:
            continue
        link = div.find("h3").a.get("href")
        match = re.search("https://[^&]+", link)
        if match is not None:
            url = match.group()
            print(url)
            if re.search("[en|hr].wikipedia", url):
                title = url.split("/")[-1]
                while True:
                    match = re.search("(%25|%)[a-fA-f0-9]{2}(%25|%)[a-fA-F0-9]{2}", title)
                    if match is None:
                        if re.search("hr.wikipedia", url):
                            url_0 = "/".join(url.split("/")[:-1])
                            url_tmp = f"{url_0}/{title}"
                            page_tmp = requests.get(url_tmp)
                            soup_tmp = BeautifulSoup(page_tmp.content, "html.parser")
                            
                            engl_link = [a.get("href") for a in soup_tmp.find_all("a", class_="interlanguage-link-target") if a.get("lang") == "en"][0]
                            match = re.search("https://[^&]+", engl_link)
                            url = match.group()
                            print(url)
                            title = url.split("/")[-1]
                        else:    
                            wikipedia_titles.add(title)
                            break
                    else:
                        corrupted = match.group()
                        if re.search("%25[a-fA-f0-9]{2}%25[a-fA-F0-9]{2}", corrupted):
                            hex_charachter = corrupted.replace("%25", "").lower()
                        else:
                            hex_charachter = corrupted.replace("%", "").lower()
                        utf_decoded = codecs.decode(hex_charachter, "hex").decode('utf-8')
                        title = title.replace(corrupted, utf_decoded)

https://hr.wikipedia.org/wiki/Marin_Dr%25C5%25BEi%25C4%2587
https://en.wikipedia.org/wiki/Marin_Dr%C5%BEi%C4%87
https://bs.wikipedia.org/wiki/Marin_Dr%25C5%25BEi%25C4%2587
https://www.lektire.hr/autor/marin-drzic/
https://nova-akropola.com/lijepe-umjetnosti/knjizevnost/marin-drzic/
https://leksikon.muzej-marindrzic.eu/zivotopis/
https://www.dubrovniksungardens.com/marin-drzic-buntovnik-s-razlogom
https://povijest.hr/nadanasnjidan/zasto-je-marin-drzic-pokopan-u-veneciji-1567/


In [52]:
wikipedia_titles

{'Marin_Držić'}

In [3]:
google_the_claim("Arnold acted in Conan")

{'Conan_the_Barbarian_(1982_film)'}

In [4]:
google_the_claim("Luis Armstrong won the superbowl.")

{'Super_Bowl_VI'}

In [5]:
google_the_claim("Leo messi won an oscar")

{'Lionel_Messi'}

In [6]:
google_the_claim("Marin Sokol won an oscar")

{'César_Award_for_Best_Actress',
 'List_of_Japanese_submissions_for_the_Academy_Award_for_Best_Foreign_Language_Film'}

In [7]:
google_the_claim("Égalité")

{'Liberté,_égalité,_fraternité'}

In [8]:
google_the_claim("Über")

{'Über'}

In [9]:
google_the_claim("Rosendals Trädgård was great")

{'Rosendals_Trädgård'}

In [10]:
google_the_claim("Marin držić")

{'Marin_Držić'}

In [11]:
google_the_claim("game of thrones")

{'Game_of_Thrones'}

In [12]:
google_the_claim("igra prijestolja")

{'Game_of_Thrones'}